### Codificación de las variables categoricas, escalado y modelo:

In [4]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix

from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

In [13]:
df_fraud_train = pd.read_csv("../data/train_df_data_preprocessing_missing_outlier.csv").drop("Unnamed: 0", axis = 1)
df_fraud_test = pd.read_csv("../data/train_df_data_preprocessing_missing_outlier.csv").drop("Unnamed: 0", axis = 1)

In [14]:
df_fraud_train.columns

Index(['intended_balcon_amount', 'prev_address_months_count',
       'bank_months_count', 'current_address_months_count',
       'session_length_in_minutes', 'device_distinct_emails_8w',
       'foreign_request', 'phone_mobile_valid', 'has_other_cards',
       'proposed_credit_limit', 'device_os', 'source', 'housing_status',
       'keep_alive_session', 'device_fraud_count', 'phone_home_valid',
       'credit_risk_score', 'email_is_free', 'income', 'employment_status',
       'date_of_birth_distinct_emails_4w', 'bank_branch_count_8w',
       'velocity_4w', 'velocity_24h', 'velocity_6h', 'zip_count_4w',
       'payment_type', 'days_since_request', 'customer_age',
       'name_email_similarity', 'month', 'fraud_bool'],
      dtype='object')

In [15]:
pd_loan_train.dtypes

intended_balcon_amount              float64
prev_address_months_count           float64
bank_months_count                   float64
current_address_months_count        float64
session_length_in_minutes           float64
device_distinct_emails_8w           float64
foreign_request                     float64
phone_mobile_valid                  float64
has_other_cards                     float64
proposed_credit_limit               float64
device_os                            object
source                               object
housing_status                       object
keep_alive_session                  float64
device_fraud_count                  float64
phone_home_valid                    float64
credit_risk_score                   float64
email_is_free                       float64
income                              float64
employment_status                    object
date_of_birth_distinct_emails_4w    float64
bank_branch_count_8w                float64
velocity_4w                     

## Codificación de la variable objetivo:

In [18]:
df_fraud_train["fraud_bool"].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

In [24]:
X_train = df_fraud_train
X_test = df_fraud_test
y_train = df_fraud_train["fraud_bool"]
y_test = df_fraud_test["fraud_bool"]

## Codificación del resto de variables categoricas:

In [26]:
categoricals_list = list(X_train.select_dtypes("object", "category").columns)
other_list = list(set(X_train.columns)-set(categoricals_list))

In [28]:
ohe = ce.OneHotEncoder(cols = categoricals_list)
model = ohe.fit(X_train, y_train)

In [29]:
model

OneHotEncoder(cols=['device_os', 'source', 'housing_status',
                    'employment_status', 'payment_type'])

In [30]:
X_train_t = model.transform(X_train, y_train)
X_test_t = model.transform(X_test, y_test)

In [35]:
X_train_t.shape

(800000, 53)

In [32]:
X_train_t

,intended_balcon_amount,prev_address_months_count,bank_months_count,current_address_months_count,session_length_in_minutes,device_distinct_emails_8w,foreign_request,phone_mobile_valid,has_other_cards,proposed_credit_limit,...,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,days_since_request,customer_age,name_email_similarity,month,fraud_bool
0,36.582496,19.000000,14.862618,13.0,9.893958,1.0,0.0,1.0,0.0,200.0,...,1,0,0,0,0,19.774313,20.0,0.430706,2.0,0
1,36.582496,28.000000,14.862618,8.0,2.206671,2.0,0.0,1.0,0.0,200.0,...,1,0,0,0,0,0.009700,30.0,0.080794,1.0,0
2,36.582496,60.719967,14.862618,145.0,3.045315,1.0,0.0,1.0,1.0,200.0,...,1,0,0,0,0,0.015138,40.0,0.614428,6.0,0
3,49.643800,60.719967,1.000000,20.0,13.025723,1.0,0.0,1.0,0.0,200.0,...,0,1,0,0,0,0.052389,30.0,0.186852,4.0,0
4,36.582496,61.000000,14.862618,15.0,1.098592,1.0,0.0,1.0,0.0,1500.0,...,1,0,0,0,0,0.013814,30.0,0.335923,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,36.582496,60.719967,11.000000,93.0,15.691726,2.0,0.0,1.0,0.0,1500.0,...,0,0,0,1,0,0.017883,50.0,0.568145,2.0,0
799996,36.582496,93.000000,11.000000,4.0,18.976904,1.0,0.0,1.0,0.0,200.0,...,0,0,0,1,0,23.095134,20.0,0.226380,6.0,0
799997,12.855974,15.000000,28.000000,17.0,15.502366,1.0,0.0,1.0,1.0,500.0,...,0,1,0,0,0,0.028021,30.0,0.504547,5.0,0
799998,38.367637,60.719967,28.000000,131.0,6.244708,1.0,0.0,1.0,0.0,200.0,...,0,1,0,0,0,0.045088,10.0,0.901936,2.0,0


In [33]:
X_train_t.dtypes.to_dict()

{'intended_balcon_amount': dtype('float64'),
 'prev_address_months_count': dtype('float64'),
 'bank_months_count': dtype('float64'),
 'current_address_months_count': dtype('float64'),
 'session_length_in_minutes': dtype('float64'),
 'device_distinct_emails_8w': dtype('float64'),
 'foreign_request': dtype('float64'),
 'phone_mobile_valid': dtype('float64'),
 'has_other_cards': dtype('float64'),
 'proposed_credit_limit': dtype('float64'),
 'device_os_1': dtype('int64'),
 'device_os_2': dtype('int64'),
 'device_os_3': dtype('int64'),
 'device_os_4': dtype('int64'),
 'device_os_5': dtype('int64'),
 'source_1': dtype('int64'),
 'source_2': dtype('int64'),
 'housing_status_1': dtype('int64'),
 'housing_status_2': dtype('int64'),
 'housing_status_3': dtype('int64'),
 'housing_status_4': dtype('int64'),
 'housing_status_5': dtype('int64'),
 'housing_status_6': dtype('int64'),
 'housing_status_7': dtype('int64'),
 'keep_alive_session': dtype('float64'),
 'device_fraud_count': dtype('float64'),


## Escalado de las variables:

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
model_scaled = scaler.fit(X_train_t)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_t), columns = X_train_t.columns, index = X_train_t.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_t), columns = X_test_t.columns, index = X_test.index)

In [37]:
X_train_scaled.describe()

,intended_balcon_amount,prev_address_months_count,bank_months_count,current_address_months_count,session_length_in_minutes,device_distinct_emails_8w,foreign_request,phone_mobile_valid,has_other_cards,proposed_credit_limit,...,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,days_since_request,customer_age,name_email_similarity,month,fraud_bool
count,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,...,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05
mean,1.215739e-15,4.643663e-16,-1.403322e-16,1.028511e-17,-4.543210e-16,2.273737e-15,-2.557954e-18,-7.843060e-17,4.860112e-17,-1.191580e-16,...,4.561684e-17,4.063416e-17,-5.640821e-17,-3.314682e-17,-3.330669e-18,-6.536993e-17,-1.909983e-16,2.225775e-16,4.263256e-19,-4.806822e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.103998e+00,-1.637414e+00,-1.392577e+00,-9.855283e-01,-9.420560e-01,-5.770758e+00,-1.612496e-01,-2.838548e+00,-5.359685e-01,-6.678156e-01,...,-5.805931e-01,-5.901517e-01,-7.671206e-01,-3.672368e-01,-1.669814e-02,-1.907449e-01,-1.969494e+00,-1.706670e+00,-1.488052e+00,-1.056018e-01
25%,3.370865e-04,1.286040e-04,-9.908248e-01,-7.587217e-01,-5.532160e-01,-1.079449e-01,-1.612496e-01,3.522928e-01,-5.359685e-01,-6.472938e-01,...,-5.805931e-01,-5.901517e-01,-7.671206e-01,-3.672368e-01,-1.669814e-02,-1.894099e-01,-1.138142e+00,-9.286783e-01,-1.035771e+00,-1.056018e-01
50%,3.370865e-04,1.286040e-04,-2.434180e-04,-3.844907e-01,-3.023935e-01,-1.079449e-01,-1.612496e-01,3.522928e-01,-5.359685e-01,-6.472938e-01,...,-5.805931e-01,-5.901517e-01,-7.671206e-01,-3.672368e-01,-1.669814e-02,-1.879280e-01,-3.067906e-01,-5.443843e-03,-1.312095e-01,-1.056018e-01
75%,3.370865e-04,1.286040e-04,1.017935e+00,4.887149e-01,1.624824e-01,-1.079449e-01,-1.612496e-01,3.522928e-01,-5.359685e-01,-3.163899e-02,...,1.722377e+00,1.694480e+00,1.303576e+00,-3.672368e-01,-1.669814e-02,-1.858513e-01,5.245610e-01,9.055900e-01,7.733522e-01,-1.056018e-01
max,6.464323e+00,9.471552e+00,1.721000e+00,3.868134e+00,9.771701e+00,5.554868e+00,6.201565e+00,3.522928e-01,1.865781e+00,3.251853e+00,...,1.722377e+00,1.694480e+00,1.303576e+00,2.723039e+00,5.988693e+01,1.403006e+01,4.681319e+00,1.752996e+00,1.677914e+00,9.469536e+00
